# Google Natural Language API Client
## Setup
[https://cloud.google.com/natural-language/docs/reference/libraries#client-libraries-install-python](https://cloud.google.com/natural-language/docs/reference/libraries#client-libraries-install-python)

Run:
```
export GOOGLE_APPLICATION_CREDENTIALS="[PATH]"
```

## Sample Code
Run the below code to make sure you have everything set up. Something should show up similar to:
```
Text: Hello, world!
Sentiment: 0.300000011921, 0.300000011921
```

In [1]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = u'Hello, world!'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
sentiment = client.analyze_sentiment(document=document).document_sentiment

print('Text: {}'.format(text))
print('Sentiment: {}, {}'.format(sentiment.score, sentiment.magnitude))

Text: Hello, world!
Sentiment: 0.30000001192092896, 0.30000001192092896


# REST API Call

Export API Key to environmental variable:
```
export GOOGLE_KEY=MY_SECRET_KEY
```

In [3]:
import gzip

META_CELLPHONE = 'Datasets/meta_Cell_Phones_and_Accessories.json.gz'

def get_documents(file_name):
    g = gzip.open(file_name, 'r')
    results = []
    for line in g:
        document = eval(line)
        results.append(document)
    return results

In [4]:
all_documents = get_documents(META_CELLPHONE)

In [9]:
import os
api_key = os.environ['GOOGLE_KEY']

In [52]:
import requests
import json

LANGUAGE_ENDPOINT = 'https://language.googleapis.com/v1/documents:analyzeEntities?key={}'.format(api_key)

def merge_title_description(document):
    title = ''
    description = ''
    if 'title' in document:
        title = document['title']
    if 'description' in document:
        description = document['description']
    return title + description

def perform_api_request(document):
    payload = {
        "document": {
            "type": "PLAIN_TEXT",
            "language": "EN",
            "content": merge_title_description(document)
        },
        "encodingType": "UTF8"
    }

    r = requests.post(endpoint, data=json.dumps(payload))
    return r.text

In [53]:
def save_api_request(document_name, r):
    file_name = 'Processed/{}.json'.format(document_name)
    with open(file_name, 'w') as outfile:
        json_data = json.dump(r, outfile)

In [54]:
import os

RUN_LIMIT = 100
def process_all_documents(all_documents):
    count = 0
    for document in all_documents:
        asin = document['asin']
        already_processed = os.path.exists('Processed/{}.json'.format(asin))
        if not already_processed:
            r = perform_api_request(document)
            save_api_request(asin, r)
            count = count + 1
            if (count is RUN_LIMIT):
                break

process_all_documents(all_documents)

In [50]:
def serialize_processed_document(asin):
    already_processed = os.path.exists('Processed/{}.json'.format(asin))
    if already_processed:
        data = json.load(open('Processed/{}.json'.format(asin), 'r'))
        return json.loads(data)
    else:
        return None

serialize_processed_document('1059777851')

{u'entities': [{u'mentions': [{u'text': {u'beginOffset': 100,
      u'content': u'window mount'},
     u'type': u'COMMON'},
    {u'text': {u'beginOffset': 379, u'content': u'window mount'},
     u'type': u'COMMON'}],
   u'metadata': {},
   u'name': u'window mount',
   u'salience': 0.29900506,
   u'type': u'OTHER'},
  {u'mentions': [{u'text': {u'beginOffset': 128, u'content': u'Direct Solar'},
     u'type': u'COMMON'},
    {u'text': {u'beginOffset': 407, u'content': u'Direct Solar'},
     u'type': u'COMMON'}],
   u'metadata': {},
   u'name': u'Direct Solar',
   u'salience': 0.22894204,
   u'type': u'OTHER'},
  {u'mentions': [{u'text': {u'beginOffset': 39, u'content': u'Tablet Charger'},
     u'type': u'PROPER'},
    {u'text': {u'beginOffset': 318, u'content': u'Tablet Charger'},
     u'type': u'PROPER'}],
   u'metadata': {u'mid': u'/m/0bh9flk',
    u'wikipedia_url': u'https://en.wikipedia.org/wiki/Tablet_computer'},
   u'name': u'Tablet Charger',
   u'salience': 0.13396543,
   u'type': 